In [3]:
!pip install kaggle

In [13]:
import os
old_name = r"/root/Untitled Folder"
new_name = r"/root/.kaggle"
os.rename(old_name, new_name)

OSError: ignored

In [14]:
!kaggle competitions download -c state-farm-distracted-driver-detection


state-farm-distracted-driver-detection.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
!unzip state-farm-distracted-driver-detection.zip

In [ ]:
import cv2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import random
import tensorflow as tf
from keras import layers, models
from keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from keras.models import load_model
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, roc_curve, roc_auc_score



In [ ]:
pic=plt.imread("/content/imgs/train/c0/img_100026.jpg")
plt.imshow(pic, cmap="gray")
print(pic.shape)

In [ ]:
DataDirectory = '/content/imgs/train'
TestDirectory= '/content/imgs/test'
Classes = ['c0','c1','c2','c3','c4','c5','c6','c7','c8','c9']
img_size = 120

In [ ]:
import concurrent.futures

def preprocess_image(image_path, img_size):
    img_array = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

    desired_size = (img_size, img_size)
    height, width = img_array.shape
    aspect_ratio = width / height

    if aspect_ratio >= 1:
        new_width = desired_size[0]
        new_height = int(new_width / aspect_ratio)
    else:
        new_height = desired_size[1]
        new_width = int(new_height * aspect_ratio)

    resized_image = cv2.resize(img_array, (new_width, new_height))

    pad_width = (desired_size[1] - new_height) // 2
    pad_height = (desired_size[0] - new_width) // 2
    padded_image = np.pad(resized_image, ((pad_width, pad_width), (pad_height, pad_height)), mode='constant', constant_values=0)

    rgb_image = cv2.cvtColor(padded_image, cv2.COLOR_GRAY2RGB)

    # Normalize pixel values to [0, 1]
    normalized_image = rgb_image / 255.0

    return normalized_image

def create_training_data(Classes, Datadirectory, img_size, max_images_per_class=1900):
    training_data = []

    with concurrent.futures.ThreadPoolExecutor() as executor:
        futures = []

        for class_num, category in enumerate(Classes):
            path = os.path.join(Datadirectory, category)
            image_count = 0

            for img in os.listdir(path):
                if image_count >= max_images_per_class:
                    break

                img_path = os.path.join(path, img)
                futures.append(executor.submit(preprocess_image, img_path, img_size))

                normalized_image = futures[-1].result()

                training_data.append([normalized_image, class_num])
                image_count += 1

    print("Data processing completed.")
    return training_data


In [ ]:
import concurrent.futures

def preprocess_image(image_path, img_size):
    img_array = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

    desired_size = (img_size, img_size)
    height, width = img_array.shape
    aspect_ratio = width / height

    if aspect_ratio >= 1:
        new_width = desired_size[0]
        new_height = int(new_width / aspect_ratio)
    else:
        new_height = desired_size[1]
        new_width = int(new_height * aspect_ratio)

    resized_image = cv2.resize(img_array, (new_width, new_height))

    pad_width = (desired_size[1] - new_height) // 2
    pad_height = (desired_size[0] - new_width) // 2
    padded_image = np.pad(resized_image, ((pad_width, pad_width), (pad_height, pad_height)), mode='constant', constant_values=0)

    rgb_image = cv2.cvtColor(padded_image, cv2.COLOR_GRAY2RGB)

    # Normalize pixel values to [0, 1]
    normalized_image = rgb_image / 255.0

    return normalized_image

def create_testing_data(TestDirectory, img_size, max_images=80_000_000):
    testing_data = []
    path = os.path.join(TestDirectory, '')
    count = 0

    with concurrent.futures.ThreadPoolExecutor() as executor:
        futures = []

        for img in os.listdir(path):
            if count >= max_images:
                break

            img_path = os.path.join(path, img)
            futures.append(executor.submit(preprocess_image, img_path, img_size))
            count += 1

        for future in concurrent.futures.as_completed(futures):
            testing_data.append([future.result()])

    print("Data processing completed.")
    return testing_data


In [ ]:
training_data= create_training_data(Classes, DataDirectory, img_size)

In [ ]:
testing_data= create_testing_data(TestDirectory, img_size)

In [ ]:
random.shuffle(training_data)

In [ ]:
print(len(training_data))
print(len(testing_data))

In [ ]:
X = []
y = []
for features, label in training_data:
    X.append(features)
    y.append(label)

X = np.array(X).reshape(-1, img_size, img_size, 3)
Y=np.array(y)


In [ ]:
# Define the input shape
input_shape = (120, 120, 3)
model = models.Sequential()

model.add(layers.Conv2D(16, (3, 3), activation='relu', input_shape=input_shape))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(32, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.BatchNormalization())  # Add batch normalization
model.add(layers.Dropout(0.5))  # Add dropout for regularization
model.add(layers.Dense(64, activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.Dropout(0.5))
model.add(layers.Dense(10, activation='softmax'))


In [ ]:
model.summary()

In [ ]:
from keras.utils import to_categorical

model.compile(loss="binary_crossentropy", optimizer = "adam", metrics = ["accuracy"])

early_stopping_callback = EarlyStopping(monitor='val_accuracy', patience=4, mode='max', verbose=1)

X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=0.2, random_state=42, shuffle=True)

Y_train = to_categorical(Y_train,num_classes=10)
Y_val = to_categorical(Y_val,num_classes=10)

In [ ]:
history = model.fit(X_train, Y_train, epochs=20, batch_size=32, validation_data=(X_val, Y_val), callbacks=[early_stopping_callback])


In [ ]:
# Plot training & validation accuracy values
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

In [ ]:
X_test = []
X_test=testing_data
print(len(X_test))
#Y_test=np.array(y_test)

In [ ]:
import numpy as np

num_parts = 10
part_size = len(X_test) // num_parts
X_test = np.array(X_test)  # Convert to NumPy array
X_test = X_test.astype('float32') / 255.0
# Process predictions in smaller batches
batch_size = 32  # Adjust this based on your available memory

all_predictions = []

for i in range(num_parts):
    start_idx = i * part_size
    end_idx = (i + 1) * part_size
    part_X_test = X_test[start_idx:end_idx]

    num_batches = len(part_X_test) // batch_size
    part_predictions = []

    for j in range(num_batches):
        batch_start = j * batch_size
        batch_end = (j + 1) * batch_size
        batch_predictions = model.predict(part_X_test[batch_start:batch_end])
        part_predictions.append(batch_predictions)

    # Process the remaining samples (not forming a complete batch)
    remaining_samples = len(part_X_test) % batch_size
    if remaining_samples > 0:
        batch_start = num_batches * batch_size
        batch_end = batch_start + remaining_samples
        batch_predictions = model.predict(part_X_test[batch_start:batch_end])
        part_predictions.append(batch_predictions)

    all_predictions.append(np.concatenate(part_predictions, axis=0))

# Concatenate all the predictions into a single array
final_predictions = np.concatenate(all_predictions, axis=0)


In [ ]:
# Create a DataFrame from the concatenated predictions
Pred = pd.DataFrame(all_predictions, columns=['c0', 'c1', 'c2', 'c3', 'c4', 'c5', 'c6', 'c7', 'c8', 'c9'])
Pred.head()

In [ ]:
sample_sub = pd.read_csv("/content/sample_submission.csv")
sample_sub.head()

In [ ]:
img = pd.read_csv("/content/driver_imgs_list.csv")
img=img['img']
img.head(20)

In [ ]:
#Pred = pd.DataFrame(predictions, columns = ['c0','c1','c2','c3','c4','c5','c6','c7','c8','c9'])
#Pred.head()

In [ ]:
sub = pd.concat((img, Pred), axis = 1)
sub.head()

In [ ]:
sub[['img', 'c0', 'c1', 'c2', 'c3', 'c4', 'c5', 'c6', 'c7', 'c8', 'c9']].to_csv(r'/content/sub0.csv', index=False)